In [1]:
%cd ..

/home/broise/tmp_projects/fb_impact


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

This notebook contains the ETL process for the Facebook dataset

## Extract

In [3]:
ds = pd.read_csv("./data/raw/dataset_Facebook.csv", sep=";")

## Transform

In [4]:
ds.head(5)

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0.0,2752,5091,178,109,159,3078,1640,119,4,79.0,17.0,100
1,139441,Status,2,12,3,10,0.0,10460,19057,1457,1361,1674,11710,6112,1108,5,130.0,29.0,164
2,139441,Photo,3,12,3,3,0.0,2413,4373,177,113,154,2812,1503,132,0,66.0,14.0,80
3,139441,Photo,2,12,2,10,1.0,50128,87991,2211,790,1119,61027,32048,1386,58,1572.0,147.0,1777
4,139441,Photo,2,12,2,3,0.0,7244,13594,671,410,580,6228,3200,396,19,325.0,49.0,393


In [5]:
## we dropna as there are not many Nan values

print(f"we found n={len(ds[ds.isnull().any(axis=1)])} rows containing NaN values")
ds_processed = ds.dropna(axis=0)
assert len(ds_processed) == len(ds) - len(ds[ds.isnull().any(axis=1)])

we found n=5 rows containing NaN values


As we want to train a model using this data, we will need to transform some of the columns. 

The Encoding changes we want to make:
+ "Type", "Category": Convert to a one hot encoded vector
+ "Post Month", "Post Weekday", "Post hour": keep ordinal encoding (For those variables, there is a form of order so I don't think it is mandatory to use one-hot encoding)
Cleaning:

For some fields, we will want to keep the values within a range (e.g. "Lifetime post consumptions", "Like", "Comment" have a long tail, maybe we can limit the max value to better represent the lower spectrum).

The scaling:
+ We will use MinMax scaler to get features value between 0 and 1.

In [6]:
ds_processed = pd.concat([pd.get_dummies(ds_processed.Category), ds_processed], axis=1)
ds_processed = pd.concat([pd.get_dummies(ds_processed.Type), ds_processed], axis=1)
ds_processed = ds_processed.rename(
    columns={1: "category_1", 2: "category_2", 3: "category_3"}
)
ds_processed = pd.concat(
    [pd.get_dummies(ds_processed["Post Weekday"]), ds_processed], axis=1
)
ds_processed = ds_processed.rename(
    columns={
        1: "day_1",
        2: "day_2",
        3: "day_3",
        4: "day_4",
        5: "day_5",
        6: "day_6",
        7: "day_7",
    }
)
ds_processed = ds_processed.drop(columns=["Type", "Category"])

In [7]:
columns_to_threshold = [
    "Page total likes",
    "Lifetime Post Total Reach",
    "Lifetime Post Total Impressions",
    "Lifetime Engaged Users",
    "Lifetime Post Consumers",
    "Lifetime Post Consumptions",
    "Lifetime Post Impressions by people who have liked your Page",
    "Lifetime Post reach by people who like your Page",
    "Lifetime People who have liked your Page and engaged with your post",
]

In [8]:
for col in columns_to_threshold:
    threshold = np.percentile(ds_processed[col], 95)
    ds_processed[col] = ds_processed.apply(lambda x: min(x[col], threshold), axis=1)

In [9]:
scaler = preprocessing.MinMaxScaler()

In [10]:
columns_to_scale = [
    "Page total likes",
    "Post Month",
    "Post Weekday",
    "Post Hour",
    "Lifetime Post Total Reach",
    "Lifetime Post Total Impressions",
    "Lifetime Engaged Users",
    "Lifetime Post Consumers",
    "Lifetime Post Consumptions",
    "Lifetime Post Impressions by people who have liked your Page",
    "Lifetime Post reach by people who like your Page",
    "Lifetime People who have liked your Page and engaged with your post",
]
ds_processed[columns_to_scale] = scaler.fit_transform(ds_processed[columns_to_scale])

In [11]:
ds_processed[
    ["Link", "Photo", "Status", "category_1", "category_2", "category_3"]
] = ds_processed[
    ["Link", "Photo", "Status", "category_1", "category_2", "category_3"]
].astype(
    "float"
)

In [12]:
ds_processed

,day_1,day_2,day_3,day_4,day_5,day_6,day_7,Link,Photo,Status,...,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,0,0,0,1,0,0,0,0.0,1.0,0.0,...,0.065415,0.043245,0.033083,0.052242,0.061668,0.060224,4,79.0,17.0,100
1,0,0,1,0,0,0,0,0.0,0.0,1.0,...,0.560480,0.584674,0.367225,0.231832,0.258091,0.601697,5,130.0,29.0,164
2,0,0,1,0,0,0,0,0.0,1.0,0.0,...,0.065028,0.044975,0.031981,0.046708,0.055650,0.067342,0,66.0,14.0,80
3,0,1,0,0,0,0,0,0.0,1.0,0.0,...,0.852332,0.337744,0.244817,1.000000,1.000000,0.753901,58,1572.0,147.0,1777
4,0,1,0,0,0,0,0,0.0,1.0,0.0,...,0.256242,0.173413,0.125937,0.117778,0.130187,0.211881,19,325.0,49.0,393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,0,0,0,0,0,0,1,0.0,1.0,0.0,...,0.310045,0.323041,0.219232,0.105836,0.131505,0.226116,10,125.0,41.0,176
495,0,0,0,0,0,0,1,0.0,1.0,0.0,...,0.280240,0.302283,0.215262,0.087028,0.115956,0.209691,5,53.0,26.0,84
496,0,0,0,0,1,0,0,0.0,1.0,0.0,...,0.204374,0.215793,0.149537,0.070613,0.082048,0.159869,0,53.0,22.0,75
497,0,0,0,0,1,0,0,0.0,1.0,0.0,...,0.238436,0.243470,0.173357,0.086862,0.094522,0.193813,4,93.0,18.0,115


## Load

In [14]:
ds_processed.to_parquet("./data/processed/dataset_facebook.parquet")